In [ ]:
%pip install optuna scikit-learn

In [30]:
import optuna
from optuna.samplers import TPESampler
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [ ]:
import pandas as pd

data=load_breast_cancer()
X=pd.DataFrame(data.data, columns=data.feature_names)
y=pd.Series(data.target, name='target')

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val=train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
models = {
    "SGD": {
        "model": SGDClassifier,
        "params": lambda trial: {
            "alpha": trial.suggest_float("alpha", 1e-5, 1e-1, log=True),
            "max_iter": trial.suggest_int("max_iter", 500, 2000, step=100)
        }
    },
    "DecisionTree": {
        "model": DecisionTreeClassifier,
        "params": lambda trial: {
            "max_depth": trial.suggest_int("max_depth", 3, 20),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10)
        }
    }
}

In [34]:
def Objective(trial):
    model_name=trial.suggest_categorical("model", list(models.keys()))
    model_class=models[model_name]["model"]
    params=models[model_name]["params"](trial)

    model=model_class(**params, random_state=42)
    model.fit(X_train, y_train)
    y_pred=model.predict(X_val)
    
    return f1_score(y_val, y_pred)

In [35]:
sample=TPESampler()
train=optuna.create_study(direction="maximize", sampler=sample)
train.optimize(Objective,n_trials=30)

[I 2025-02-11 13:58:38,915] A new study created in memory with name: no-name-3fc319f1-8712-4414-a8c8-6886c3a6c0cd
[I 2025-02-11 13:58:38,943] Trial 0 finished with value: 0.951048951048951 and parameters: {'model': 'DecisionTree', 'max_depth': 8, 'min_samples_split': 5}. Best is trial 0 with value: 0.951048951048951.
[I 2025-02-11 13:58:38,996] Trial 1 finished with value: 0.965034965034965 and parameters: {'model': 'SGD', 'alpha': 0.0004456501303138103, 'max_iter': 1000}. Best is trial 1 with value: 0.965034965034965.
[I 2025-02-11 13:58:39,010] Trial 2 finished with value: 0.864 and parameters: {'model': 'SGD', 'alpha': 0.02419951891753162, 'max_iter': 1800}. Best is trial 1 with value: 0.965034965034965.
[I 2025-02-11 13:58:39,038] Trial 3 finished with value: 0.9583333333333334 and parameters: {'model': 'DecisionTree', 'max_depth': 3, 'min_samples_split': 6}. Best is trial 1 with value: 0.965034965034965.
[I 2025-02-11 13:58:39,058] Trial 4 finished with value: 0.9577464788732394 a

In [36]:
print("Best Model & Hyperparameters:", train.best_params)
print("Best score:", train.best_value)

Best Model & Hyperparameters: {'model': 'SGD', 'alpha': 0.08187648189041259, 'max_iter': 2000}
Best score: 0.9793103448275862
